# Text generation: GPT-2
Conditional text generation with the auto-regressive models of the library (GPT/GPT-2)

## License

In [ ]:
# Copyright 2018 Google AI, Google Brain and Carnegie Mellon University Authors and the HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Let's get started !

In [ ]:
# coding=utf-8

import logging
from tqdm import trange

import torch
import torch.nn.functional as F
import numpy as np

!pip install transformers

from transformers import GPT2Config
from transformers import GPT2LMHeadModel, GPT2Tokenizer

Hardcoded max length to avoid infinite loop

In [ ]:
MAX_LENGTH = int(10000)

Padding text to help Transformer-XL and XLNet with short prompts as proposed by Aman Rusia in https://github.com/rusiaaman/XLNet-gen#methodology and https://medium.com/@amanrusia/xlnet-speaks-comparison-to-gpt-2-ea1a4e9ba39e

In [ ]:
PADDING_TEXT = """ In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing. <eod> </s> <eos>"""

## Helper functions



In [ ]:
def set_seed(args):
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

The function `top_k_top_p_filtering` does the following:

Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        
Args:
* logits: logits distribution shape (batch size x vocabulary size)
* top_k > 0: keep only top k tokens with highest probability (top-k filtering).
* top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                
Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317

In [ ]:
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ 
    """
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        # scatter sorted tensors to original indexing
        indices_to_remove = sorted_indices_to_remove.scatter(dim=1, index=sorted_indices, src=sorted_indices_to_remove)
        logits[indices_to_remove] = filter_value
    return logits

## function `sample_sequence`

In [ ]:
def sample_sequence(model, length, context, num_samples=1, temperature=1, top_k=0, top_p=0.0, repetition_penalty=1.0,
                    device='cpu'):
    context = torch.tensor(context, dtype=torch.long, device=device)
    context = context.unsqueeze(0).repeat(num_samples, 1)
    generated = context
    with torch.no_grad():
        for _ in trange(length):

            inputs = {'input_ids': generated}
            outputs = model(**inputs)  # Note: we could also use 'past' with GPT-2 (cached hidden-states)
            next_token_logits = outputs[0][:, -1, :] / (temperature if temperature > 0 else 1.)

            # repetition penalty from CTRL (https://arxiv.org/abs/1909.05858)
            for i in range(num_samples):
                for _ in set(generated[i].tolist()):
                    next_token_logits[i, _] /= repetition_penalty
                
            filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
            if temperature == 0: # greedy sampling:
                next_token = torch.argmax(filtered_logits, dim=-1).unsqueeze(-1)
            else:
                next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_token), dim=1)
    return generated

# Main function: `generate_text`

In [ ]:
def generate_text(raw_text, 
                  prompt="", 
                  padding_text="", 
                  words=50, 
                  num_samples=1, 
                  temperature=1.0, 
                  repetition_penalty=1.0, 
                  top_k=0, 
                  top_p=0.9, 
                  no_cuda=False, 
                  seed=42, 
                  model_path="/mnt/usb/bodels"):

    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    n_gpu = torch.cuda.device_count()

    #set_seed(1337)

    # this can be changed to simply "gpt2-medium" to download a new copy of the model state
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
    model = GPT2LMHeadModel.from_pretrained("gpt2-large")
    model.to(device)
    model.eval()

    if words < 0 and model.config.max_position_embeddings > 0:
        words = model.config.max_position_embeddings
    elif 0 < model.config.max_position_embeddings < words:
        words = model.config.max_position_embeddings  # No generation bigger than model size 
    elif words < 0:
        words = MAX_LENGTH  # avoid infinite loop

    context_tokens = tokenizer.encode(raw_text, add_special_tokens=False)
    out = sample_sequence(
        model=model,
        context=context_tokens,
        num_samples=num_samples,
        length=words,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        device=device,
    )
    out = out[:, len(context_tokens):].tolist()
    print("\n"+ raw_text)
    for o in out:
        text = tokenizer.decode(o, clean_up_tokenization_spaces=True)
    return text

In [ ]:
generate_text("slowly we rounded the corner, exclaiming to the dragon that we could if only we wanted ")